## **Samples: Multi-AI Agents for Booking hotel**

For today fast-paced world, to plan business trip no be only to book flight and hotel room. E need coordination and efficiency level wey fit hard to achieve. Na here Multi-AI Agents come dey play, to change how we dey manage our travel needs.

Imagine say you get team of smart agents wey dey work together to handle every part of your trip with correct accuracy and ease. With our advanced AI technology, we don create special agents for booking services and itinerary arrangement, to make sure say travel experience smooth and stress-free.

This one na basic scenario. When you dey plan business trip, you go need make business travel agent help you find air ticket information, hotel information, and others. Through AI Agents, we fit build agents for booking services and agents for itinerary arrangement wey go collaborate and improve how smart the arrangement go be.


# Initialize di Azure AI Agent Service and get configuration info from **.env**

### **.env** 

Create one .env file 

**.env** get di connection string for Azure AI Agent Service, di model wey AOAI dey use, and di correct Google API Search service API, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Your Azure AI Agent Service Model Deployment Name"

[**NOTE**] You go need model wey get 100,000 Rate Limit (Tokens per minute) Rate Limit of 600 (Request per minute)

  You fit find model for Microsoft Foundry - Model and Endpoint. 


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Your Azure AI Agent Service Project Connection String"

  You fit find di project connection string for your project overview for AI ​​Foundry Portal Screen.

- **SERPAPI_SEARCH_API_KEY** = "Your SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Your SERPAPI Search Endpoint"

To get di Model Deployment Name and Project Connection String of Azure AI Agent Service, you go need create Azure AI Agent Service. E good to use [this template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) to create am directly （***Note:*** Azure AI Agent Service dey currently set for limited region. E good make you check [this link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) to set the region)

Agent need access SERPAPI. E good make you sign up using [this link](https://serpapi.com/searches). After you register, you fit get unique API KEY and ENDPOINT


# Login to Azure 

You need to login to Azure now. Open terminal for VScode and run the `az login` command


# Setup 

To run dis notebook, you go need install dis kind libraries dem. Na dis libraries dem and di pip install commands wey go follow: 

azure-identity: For Azure authentication.  
requests: For making HTTP requests.  
semantic-kernel: For di semantic kernel framework (if na custom or specific library dis be, you fit need install am from specific source or repository).


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Explanation: 
import asyncio: Dis one dey import asyncio module, wey dey give support for asynchronous programming for Python. E allow you write concurrent code using the async and await syntax.
from typing import Annotated: Dis one dey import the Annotated type from the typing module. Annotated dey used to add metadata to type hints, wey fit useful for different kain tins like validation, documentation, or tooling


In [ ]:
import asyncio,os
from typing import Annotated

# Explanation:
By using from dotenv import load_dotenv and load_dotenv(), you fit easily manage configuration settings and sensitive information (like API keys and database URLs) for one .env file, keep dem separate from your source code and make your application more secure and easier to configure.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Explanation:

Import Statement: from azure.identity.aio import DefaultAzureCredential: Dis one dey import di DefaultAzureCredential class from di azure.identity.aio module. Di aio part for di module name mean say e dey meant for asynchronous operations.

Purpose of DefaultAzureCredential: Di DefaultAzureCredential class na part of di Azure SDK for Python. E dey provide one default way to authenticate wit Azure services. E dey try authenticate using different methods for one correct order, like environment variables, managed identity, and Azure CLI credentials.

Asynchronous Operations: Di aio module mean say di DefaultAzureCredential class support asynchronous operations. Dis one mean say you fit use am wit asyncio to perform non-blocking authentication requests.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Explanation:
Dem dey import different modules and classes from the semantic_kernel package. Below na wetin each import mean:

AgentGroupChat from semantic_kernel.agents: Dis class dey handle functionalities wey concern group chat for AI agents. AzureAIAgent and AzureAIAgentSettings from semantic_kernel.agents.azure_ai

AzureAIAgent: Dis class dey used to create and manage AI agents wey dey use Azure AI services.

AzureAIAgentSettings: Dis class dey used to configure settings for the AzureAIAgent. TerminationStrategy from semantic_kernel.agents.strategies.termination.termination_strategy:

Dis class dey define strategies wey fit terminate execution of AI agents under certain conditions. ChatMessageContent from semantic_kernel.contents.chat_message_content:

Dis class dey used to handle the content of chat messages.
AuthorRole from semantic_kernel.contents.utils.author_role:

Dis class dey define different roles for authors inside the context of chat messages.

kernel_function from semantic_kernel.functions.kernel_function_decorator: Dis decorator dey used to define kernel functions, wey be functions wey fit run inside the semantic kernel framework.
All dis imports dey set up the necessary components for to create and manage AI agents wey fit interact for group chat environment, fit be for tasks like booking hotels or similar activities.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explanation:
Next we import the CodeInterpreterTool class from the azure.ai.projects.models module. 

CodeInterpreterTool: Dis class na part of the Azure AI SDK and e dey used to interpret and run code inside AI projects context. E dey provide functionalities for running code snippets, analyzing code, or to join code execution inside AI workflows.
Dis import dey prepare the necessary component to use the CodeInterpreterTool for your project, we fit use am for tasks wey involve interpreting and running code dynamically.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Explanation: 
The ApprovalTerminationStrategy class dey provide one specific strategy to stop how AI agent dey work. The agent go stop if the last message for im interaction history get the word "saved". This fit help for situations where the agent task don finish once e receive confirmation say something don "saved".Define the interaction method. After the reservation plan don save, e fit stop when e see the saved signal.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Explanation:

Di line of code dey initialize one AzureAIAgentSettings object wit default or predefined settings by to call di create() method. Dis settings object (ai_agent_settings) fit den be use to configure and manage one AzureAIAgent instance.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Explanation:
By importing the requests library, you fit easy make HTTP requests and interact with web services for your Python code.


In [ ]:
import requests

# Explanation:
Dis na variable wey dey store di API key wey you go take access SERP (Search Engine Results Page) API service. API key na unique identifier wey dem dey use confirm say di request na from your account.

'GOOGLE_SEARCH_API_KEY': Dis na placeholder string. You go need comot 'GOOGLE_SEARCH_API_KEY' put your correct SERP API key.

Purpose: Di purpose of dis line na to store di API key for one variable so dat e fit dey use confirm requests wey dem dey send to di SERP API service. You need di API key take access di service and run searches.

How to Get a SERP API Key: To get SERP API key, follow dis general steps for https://serpapi.com (di exact steps fit different depending on di particular SERP API service wey you dey use):

Choose a SERP API Service: Plenty SERP API services dey available, like SerpAPI, Google Custom Search JSON API, and others. Choose di one wey beta for your needs.

Sign Up for an Account:

Go di website of di SERP API service wey you choose https://www.serpapi.com and sign up for account. You fit need give some basic info and confirm your email address.

Create an API Key:

After you don sign up, log in your account and go the API section or dashboard. Find option to create or generate new API key.

Copy the API Key:

As soon as your API key don generate, copy am. Dis key na wetin you go use take confirm your requests to di SERP API service.

Replace the Placeholder:

Replace di placeholder inside your .env file


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Explanation:
Di BookingPlugin class dey provide methods for booking hotels and flights by using di Serpapi.com Google Search API. E dey construct di necessary parameters, send API requests, and process di responses to return relevant booking information. Di API key (SERPAPI_SEARCH_API_KEY) and endpoint (SERPAPI_SEARCH_ENDPOINT) na dem dey use authenticate and send requests to di Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Explanation:
Di SavePlugin class dey provide one method wey e call saving_plan to save trip plans using Azure AI services. E dey set up Azure credentials, create one AI agent, process user inputs to generate and save di trip plan content, and e dey handle file saving and cleanup operations. Di method go return "Saved" when e don complete well.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Explanation:
Dis code dey set up Azure AI agents to handle booking flights and hotels, and to save trip plans based on wetin user talk. E dey use Azure credentials to create and arrange the agents, e dey run user inputs through group chat, and e dey make sure say everything clean after the tasks don finish. The agents dey use certain plugins (BookingPlugin and SavePlugin) to do their own work.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:
Dis document don translate wit AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). Even though we dey try make am correct, abeg sabi say automated translation fit get some errors or mistakes. Di original document wey dem write for dia own language na di correct one wey you suppose trust. If information na serious one, e better make professional human person translate am. We no go responsible for any wahala or wrong meaning wey fit happen because of dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
